## Imports
utilities.py contains a number of useful functions including loading credentials from a json file, retrieving files from S3, and defining thecurrent list of brains that are associated with the Active Atlas.

In [1]:
import datajoint as dj  # automatically loads dj.config from the file `dj_local_conf.json` if it exists
import numpy as np
import json
from subprocess import call
import yaml
import sys, os
import pandas as pd

sys.path.append('./lib')
from utilities import *
from initialization_of_db import *

## Load Credentials, Connect to Database
User needs to supply credentials to connect to the dj server and to S3.

1. The master file is in `setup/credFiles.yaml`. This file contains pointers to the dj and S3 credentials. A typical contant is 
``` yaml
aws_fp: /Users/yoavfreund/VaultBrain/s3-creds.json
dj_fp: /Users/yoavfreund/VaultBrain/dj_local_conf.json
```

2. A redacted content of the json file `aws_fp` is:
```json
{"access_key": "XXXX", 
"secret_key": "SSSSSSS"}
```

3. A redacted content of the json file `dj_fp` is 
```json
{
    "database.host": "ucsd-demo-db.datajoint.io",
    "database.password": "XXXXXXXXXXX",
    "database.user": "yoav",
    "database.port": 3306,
    "database.reconnect": false,
    "connection.init_function": null,
    "connection.charset": "",
    "loglevel": "INFO",
    "safemode": true,
    "display.limit": 7,
    "display.width": 14,
    "display.show_tuple_count": true,
    "history": [
        "No config file found, using default settings.",
        "Updated from config file: dj_local_conf.json"
    ]
}
```

In [2]:
%%writefile /Users/yoavfreund/VaultBrain/credFiles.yaml
aws_fp: /Users/yoavfreund/VaultBrain/s3-creds.json
dj_fp: /Users/yoavfreund/VaultBrain/dj_local_conf.json
    

Overwriting /Users/yoavfreund/VaultBrain/credFiles.yaml


In [3]:
# Load Datajoint Credentials
# `dj_creds` needs the following fields: 'user', 'passwd'
# Then set necessary fields in dj.config

credFiles='/Users/yoavfreund/VaultBrain/credFiles.yaml'
# Cannot directly set 'dj.config' equal to the json file as it is a special datajoint object
dj.config['database.host'] = get_dj_creds(credFiles)['database.host']
dj.config['database.user'] = get_dj_creds(credFiles)['database.user']
dj.config['database.port'] = get_dj_creds(credFiles)['database.port']
dj.config['database.password'] = get_dj_creds(credFiles)['database.password']

# Will save a json file in this directory (ignored by git) with credentials. Won't need to load them again
dj.config.save_local()

In [4]:
# Connect to datajoint server
dj.conn()

Connecting yoav@ucsd-demo-db.datajoint.io:3306


DataJoint connection (connected) yoav@ucsd-demo-db.datajoint.io:3306

In [5]:
# Define which schema you're using
schema = dj.schema('common_atlas_v3')
schema.spawn_missing_classes()

#dj.ERD(schema)

In [6]:
print('________ Mouse ________')
Mouse.describe()
print('________ Histology ________')
Histology.describe()
print('________ Stack ________')
Stack.describe()
print('________ Slice ________')
Slice.describe();

________ Mouse ________
mouse                : char(18)                     # Name for lab mouse, max 8 chars
---
date_of_birth        : date                         # (date) the mouse's date of birth
sex                  : enum('M','F')                # (M/F) either 'M' for male, 'F' for female
genotype             : enum('C57','U')              # (Lookup) indicating the genotype
weight               : double                       # (int) weight of the mouse in grams. -1 if unknown
bred                 : varchar(20)                  # (Str) Vendor where the mouse was bred (bred in house,     #purchased by vendor)

________ Histology ________
-> Mouse
---
region               : varchar(10)                  # (Str) [UNSURE]
thickness            : int                          # (int) thickness of each slice in microns
orientation          : enum('sagittal','coronal','horozontal') # (Str) horizontal, sagittal, coronal
counter_stain        : varchar(30)                  # (Str) what stain 

## extracting rows as pandas dataframe

In [7]:
brainStackTable = Stack.fetch(as_dict=True)
brainStackDF = pd.DataFrame(brainStackTable)
brainStackDF.head()

,mouse,stack_name,num_slices,num_valid_slices,channels,human_annotated,planar_resolution_um,section_thickness_um
0,CHATM2,CHATM2,328,235,2,0,0.46,20.0
1,CHATM3,CHATM3,413,293,2,0,0.46,20.0
2,CSHL2,CSHL2,-1,-1,1,0,0.46,20.0
3,MD175,MD175,-1,-1,1,0,0.46,20.0
4,MD585,MD585,447,439,1,1,0.46,20.0


In [8]:
HistologyTable = Histology.fetch( as_dict=True )
HistologyDF=pd.DataFrame.from_dict(HistologyTable)
print(HistologyDF.shape)
HistologyDF.head()

(27, 7)


,mouse,region,thickness,orientation,counter_stain,lab,series
0,CHATM2,Unknown,20,sagittal,NTB/ChAT,UCSD,unknown
1,CHATM3,Unknown,20,sagittal,NTB/ChAT,UCSD,unknown
2,CSHL2,Unknown,20,sagittal,?,UCSD,unknown
3,MD175,Unknown,20,coronal,thionin,CSHL,unknown
4,MD585,Unknown,20,sagittal,thionin,CSHL,unknown


## List relevant info for given stack
#### - Example displaying MD585 sections


In [9]:
stack = 'MD585'
ThisBrain=brainStackDF[brainStackDF['mouse']==stack]
ThisBrain[['mouse','stack_name','num_slices','channels']]

,mouse,stack_name,num_slices,channels
4,MD585,MD585,447,1


In [10]:
ThisHistology=HistologyDF[HistologyDF['mouse']==stack]
ThisHistology

,mouse,region,thickness,orientation,counter_stain,lab,series
4,MD585,Unknown,20,sagittal,thionin,CSHL,unknown


### Define a computed table

With an entry for each slice.

In [11]:
@schema
class Process(dj.Computed):
    definition="""
    -> Slice
    ---
    size:int #size of raw file
    """
    bucket=os.environ['BUCKET_RAWDATA']
    credFiles='/Users/yoavfreund/VaultBrain/credFiles.yaml'
    client=get_s3_client(credFiles)
    def _make_tuples(self,key):
        print('populating for',key)
        raw_s3_fp = (Slice & key).fetch1('raw_s3_fp')
        if len(raw_s3_fp)>0:
            print('raw_s3_fp=',raw_s3_fp,end='')
            report=client.stat_object(bucket,raw_s3_fp)
            key['size']=report.size
            print('size=',report.size)
            self.insert1(key)

In [12]:
sizes=Process()
sizes.populate()
sizes


populating for {'mouse': 'CHATM2', 'slice_num': 1}
populating for {'mouse': 'CHATM2', 'slice_num': 2}
populating for {'mouse': 'CHATM2', 'slice_num': 3}
populating for {'mouse': 'CHATM2', 'slice_num': 4}
populating for {'mouse': 'CHATM2', 'slice_num': 5}
populating for {'mouse': 'CHATM2', 'slice_num': 6}
populating for {'mouse': 'CHATM2', 'slice_num': 7}
populating for {'mouse': 'CHATM2', 'slice_num': 8}
populating for {'mouse': 'CHATM2', 'slice_num': 9}
populating for {'mouse': 'CHATM2', 'slice_num': 10}
populating for {'mouse': 'CHATM2', 'slice_num': 11}
populating for {'mouse': 'CHATM2', 'slice_num': 12}
populating for {'mouse': 'CHATM2', 'slice_num': 13}
populating for {'mouse': 'CHATM2', 'slice_num': 14}
populating for {'mouse': 'CHATM2', 'slice_num': 15}
populating for {'mouse': 'CHATM2', 'slice_num': 16}
populating for {'mouse': 'CHATM2', 'slice_num': 17}
populating for {'mouse': 'CHATM2', 'slice_num': 18}
populating for {'mouse': 'CHATM2', 'slice_num': 19}
populating for {'mous

KeyboardInterrupt: 

In [101]:
dump=(Process.fetch( as_dict=True ))

KeyError: 'nullable'

In [93]:
!aws s3 ls s3://$bucket/$raw_s3_fp

2019-01-01 23:32:02  125686663 MD585-N3-2015.07.16-19.48.34_MD585_1_0007_lossless.jp2
